##Bibliotecas

In [ ]:
!pip install openpyxl

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from time import sleep
from datetime import datetime

#Atividade 1

In [ ]:
# Lê o conteúdo do arquivo HTML
with open('exemplo.html', 'r') as arquivo:
 conteudo = arquivo.read()
# Cria um objeto BeautifulSoup
soup = BeautifulSoup(conteudo, 'html.parser')

In [ ]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="pt-br">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <title>
   Portal de Videogames
  </title>
 </head>
 <body>
  <header>
   <h1>
    Bem-vindo ao Portal de Videogames
   </h1>
   <nav>
    <ul>
     <li>
      <a href="#ultimas-noticias">
       Últimas Notícias
      </a>
     </li>
     <li>
      <a href="#analises">
       Análises
      </a>
     </li>
     <li>
      <a href="#videos">
       Vídeos
      </a>
     </li>
    </ul>
   </nav>
  </header>
  <section id="ultimas-noticias">
   <h2>
    Últimas Notícias
   </h2>
   <article>
    <h3>
     Novo Lançamento: "Cyberpunk 2077"
    </h3>
    <p>
     O altamente aguardado jogo "Cyberpunk 2077" já está disponível para compra. Confira as análises!
    </p>
   </article>
   <article>
    <h3>
     Atualização de "Fortnite"
    </h3>
    <p>
     A mais recente atualização de "Fortnite" traz novos modos de jogo e skins exclusivas.
   

In [ ]:
article = (soup.find_all('article'))
print(article)

[<article>
<h3>Novo Lançamento: "Cyberpunk 2077"</h3>
<p>O altamente aguardado jogo "Cyberpunk 2077" já está disponível para compra. Confira as análises!</p>
</article>, <article>
<h3>Atualização de "Fortnite"</h3>
<p>A mais recente atualização de "Fortnite" traz novos modos de jogo e skins exclusivas.</p>
</article>, <article>
<h3>Análise de "The Last of Us Part II"</h3>
<p>Confira nossa análise completa do jogo "The Last of Us Part II" e descubra por que ele é um sucesso.</p>
</article>, <article>
<h3>Análise de "Assassin's Creed Valhalla"</h3>
<p>Saiba o que achamos do jogo "Assassin's Creed Valhalla" em nossa análise detalhada.</p>
</article>, <article>
<h3>Gameplay de "Red Dead Redemption 2"</h3>
<p>Assista ao nosso gameplay exclusivo de "Red Dead Redemption 2" e veja como o jogo se destaca.</p>
</article>, <article>
<h3>Entrevista com Desenvolvedores de "Halo Infinite"</h3>
<p>Confira nossa entrevista exclusiva com os desenvolvedores de "Halo Infinite" e saiba mais sobre o jogo.<

In [ ]:
p = soup.find_all('p')
for i in p:
  print(i.text + '\n')

O altamente aguardado jogo "Cyberpunk 2077" já está disponível para compra. Confira as análises!

A mais recente atualização de "Fortnite" traz novos modos de jogo e skins exclusivas.

Confira nossa análise completa do jogo "The Last of Us Part II" e descubra por que ele é um sucesso.

Saiba o que achamos do jogo "Assassin's Creed Valhalla" em nossa análise detalhada.

Assista ao nosso gameplay exclusivo de "Red Dead Redemption 2" e veja como o jogo se destaca.

Confira nossa entrevista exclusiva com os desenvolvedores de "Halo Infinite" e saiba mais sobre o jogo.

© 2023 Portal de Videogames



#Atividade 2

Crie um web scraping da página institucional
https://institucional.ufpel.edu.br/unidades/id/443

● Para capturar:

● Cursos de Graduação -> Turmas Ofertadas -> Nome das Disciplinas, Nome dos Professores,
Número de Vagas Ofertadas e Número de Alunos Matriculados

● Na sequência, apresente essas informações em um dataframe.

● E para capturar:

● Cursos de Graduação da Computação (Ciência e Engenharia) -> Professores com lotação no
Centro de Desenvolvimento Tecnológico -> Projetos em andamento de Pesquisa, Ensino e
Extensão -> Resumo

● Na sequência, salve essas informações em um arquivo csv


In [ ]:
url = 'https://institucional.ufpel.edu.br/unidades/id/443'

url_base = 'https://institucional.ufpel.edu.br'

###Capturar o código fonte da pagina

In [ ]:
def return_soup(url):
  response = requests.get(url)

  if response.status_code == 200:
      html = response.text

  soup = BeautifulSoup(html, 'html.parser')

  return soup

###Capturar URLS de cada curso

In [ ]:
def getUrl(url):

  soup = return_soup(url)

  codigo = soup.find_all('div', id='graduacao')

  links = codigo[0].find_all('a')

  href = []

  for link in links:
      href.append(link.get('href'))

  href = href[1:]
  print(href)

  url_acess = []

  for i in href:
    url_acess.append(url_base + i)

  return url_acess

###Capturar de informações das disciplinas do CDTEC

In [ ]:
def getDisciplines(url):

  soup = return_soup(url)

  versoes = soup.find_all('div', class_="versao")

  discipline = []

  for getDiv in versoes:

    ofertas = getDiv.find_all('div', class_="accordion cor-borda")

    for getTable in ofertas:

        auxDiscipline = getTable.find_all('table', class_="tabela-dados cor-borda")

        for getTdbody in auxDiscipline:

            trSoup = getTdbody.find_all('tr')

            for tdGet in trSoup:
                  discipline.append(tdGet)
  return discipline

###Raspagem das informações do HTML

In [ ]:
# Cursos de Graduação ->
# Turmas Ofertadas ->
# Nome das Disciplinas,
# Nome dos Professores,
# Número de Vagas Ofertadas
# Número de Alunos Matriculados

texto_inicial = '<span class="tabela-detalhe-info">'
texto_final = '</span>'

dados = {'jsons':[]}

def get_info_disciplines(soupInfo):
  counter = 0
  nome_disciplina = []
  professores = []
  vagas = []
  matriculados = []
  for info in soupInfo:
      aux_name = info.find('a')
      if aux_name:
          name = aux_name.text.strip()
          nome_disciplina.append(name)


      professores_span = info.find_all('span', class_="tabela-detalhe-info")

      for professor in professores_span:
          aux_professor = str(professor).split('<br/>')
          for i in range(len(aux_professor)):
            if aux_professor[i].startswith(texto_inicial):
              aux_professor[i] = aux_professor[i][len(texto_inicial):]

            if aux_professor[i].endswith(texto_final):
                aux_professor[i] = aux_professor[i][:-len(texto_final)]
            if aux_professor[i] == '':
                aux_professor.pop(i)

          professores.append(aux_professor)

      counter = counter + 1
      if counter == 3:
          try:
              vagas.append(int(info.text.strip()))
          except ValueError:
              vagas.append(None)
      elif counter == 4:
          try:
              matriculados.append(int(info.text.strip()))
          except ValueError:
              matriculados.append(None)
  dados = {
      'disciplina': nome_disciplina,
      'professores': professores,
      'vagas': vagas,
      'matriculados': matriculados
    }

    # Criando o DataFrame
  df = pd.DataFrame(dados)
  return df

###Testes

In [ ]:
url_acess = getUrl(url)

disciplines = []

for i in url_acess:
  disciplines.append(getDisciplines(i))

['/cursos/cod/5700', '/cursos/cod/3900', '/cursos/cod/3910', '/cursos/cod/6100', '/cursos/cod/6400']


In [ ]:
for i in disciplines:
  for w in i:
    print(w)

<tr><th>Disciplina / Professores / Horários</th><th>Turma</th><th>Vagas</th><th>Matric.</th></tr>
<tr><td><a href="/disciplinas/cod/22000003">22000003 - BIOLOGIA CELULAR</a><span class="tabela-detalhe-info">Professor responsável pela turma: MARTA GONCALVES AMARAL<br/>Professor Regente: PRISCILA MARQUES MOURA DE LEON<br/>Professor Regente: THAIS LARRÉ OLIVEIRA BOHN<br/>Professor Regente: VINICIUS FARIAS CAMPOS<br/></span><div class="tooltip"><a href="#"><i aria-hidden="true" class="fas fa-plus-square"></i> Horários</a><div class="tooltip-container"><table class="grade-horarios cor-borda"><tr class="cor-fundo"><th>Manhã</th><th>Tarde</th><th>Noite</th></tr><tr><td><span class="grade-horarios-dia">TER</span>08:00 - 08:50<br/><span class="grade-horarios-dia"></span>08:50 - 09:40<br/><span class="grade-horarios-dia">QUA</span>08:00 - 08:50<br/><span class="grade-horarios-dia"></span>08:50 - 09:40<br/></td><td></td><td></td></tr></table></div></div></td><td>M1</td><td>24</td><td>21</td></tr>

###Coleta dos Dados

In [ ]:
df_final = pd.DataFrame()

url_acess = getUrl(url)

disciplines = []

for i in url_acess:
  disciplines.append(getDisciplines(i))

for get in disciplines:
  for diciplina in get:
    if diciplina.find('a', href=lambda href: href and '/disciplinas/' in href):
      df_temp = get_info_disciplines(diciplina)
      df_final = pd.concat([df_final, df_temp], ignore_index=True)

# Exibir o DataFrame final
print(df_final)
df_final.to_csv('disciplinas.csv', index=False)
df_final.to_excel('disciplinas.xlsx', index=False, engine='openpyxl')

['/cursos/cod/5700', '/cursos/cod/3900', '/cursos/cod/3910', '/cursos/cod/6100', '/cursos/cod/6400']
                                            disciplina  \
0                          22000003 - BIOLOGIA CELULAR   
1                          22000003 - BIOLOGIA CELULAR   
2                    22000004 - TÉCNICAS INSTRUMENTAIS   
3    22000022 - BIOESTATÍSTICA E DELINIAMENTO EXPER...   
4                       22000021 - OPERAÇÕES UNITÁRIAS   
..                                                 ...   
398                      22000542 - PORTOS E HIDROVIAS   
399        22000545 - TRABALHO DE CONCLUSÃO DE CURSO I   
400          22000549 - ESTÁGIO CURRICULAR OBRIGATÓRIO   
401       22000548 - TRABALHO DE CONCLUSÃO DE CURSO II   
402                      22000552 - GESTÃO DE RESÍDUOS   

                                           professores  vagas  matriculados  
0    [Professor responsável pela turma: MARTA GONCA...     24            21  
1    [Professor responsável pela turma: MARTA 

#Atividade 3 - Coleta dos dados Professores UFPEL

##Configuração Selenium

In [ ]:
!pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementNotInteractableException

In [ ]:
!apt-get update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 2s (143 kB/s)
Reading package lists... Done


In [ ]:
!apt-get install chromium-driver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'chromium-chromedriver' instead of 'chromium-driver'
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 44 not upgraded.


In [ ]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

##Scraping

###Cria o browser para coleta dos dados

In [ ]:
driver = web_driver()

url = 'https://institucional.ufpel.edu.br/unidades/id/443'

url_base = 'https://institucional.ufpel.edu.br'

###Capturar o código fonte da pagina

In [ ]:
def return_soup(url):
  response = requests.get(url)

  if response.status_code == 200:
      html = response.text

  soup = BeautifulSoup(html, 'html.parser')

  return soup

###Capturar URLS de cada curso

In [ ]:
def get_url(url):

  soup = return_soup(url)

  codigo = soup.find_all('div', id='graduacao')

  links = codigo[0].find_all('a')

  href = []

  for link in links:
      href.append(link.get('href'))

  href = href[1:]

  url_acess = []

  for i in href:
    url_acess.append(url_base + i)

  return url_acess

###Capturar URLS de cada professor dos cursos buscados

In [ ]:
def get_professor(url):
  driver.get(url)
  href = []
  first_button_id = 'prof-sup'
  second_button_id = 'DataTables_Table_2_next'
  xpath_pagination = '/html/body/div/article/div[1]/div[3]/div[3]/div/div[4]/span'
  sleep(3)

  try:
        first_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, first_button_id))
        )
        driver.execute_script("arguments[0].scrollIntoView();", first_button)
        first_button.click()

  except TimeoutException:
        print("Timeout: Um dos botões não ficou disponível para clique.")
  except ElementNotInteractableException:
        print("Erro: Um dos botões não está interagível no momento.")

  pagination_element = driver.find_element(By.XPATH, xpath_pagination)

  pagination_soup = BeautifulSoup(pagination_element.get_attribute('outerHTML'), 'html.parser')

  paginate_buttons = pagination_soup.select('a.paginate_button')
  max_value = max(int(button['data-dt-idx']) for button in paginate_buttons)

  for index in range(max_value):

    try:
        page_source = driver.page_source

        soup = BeautifulSoup(page_source, 'html.parser')

        tableProfessor = soup.find('table', class_='tabela-dados tabela-listagem-professor tabela-com-cursor tabela-grid tabela-grid-3 cor-borda dataTable no-footer')

        tableProfessor = soup.find_all('div', class_='servidor-foto cor-fundo')

        for a in tableProfessor:

            href.append(a.find('a').get('href'))


        second_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, second_button_id))
        )
        driver.execute_script("arguments[0].scrollIntoView();", second_button)
        second_button.click()

    except TimeoutException:
        print("Timeout: Um dos botões não ficou disponível para clique.")
    except ElementNotInteractableException:
        print("Erro: Um dos botões não está interagível no momento.")


  url_acess = []

  for i in href:
    url_acess.append(url_base + i)

  return url_acess


###Coletar dados referente a enfase do projeto e resumo do projeto

In [ ]:
def get_informacao(url):

  div_resume = '/html/body/div/article/div[1]/div[1]/div[14]'

  request = requests.get(url)

  if request.status_code == 200:
      html = request.text

  soup = BeautifulSoup(html, 'html.parser')

  resume = soup.find_all('div', class_='ficha-campo')

  return resume[1].text, resume[-1].text

###Coletar as informações referente a cada projeto que o professor buscado participou ou participa

In [ ]:
def get_info_professor(url):

  driver.get(url)

  page_source = driver.page_source

  soup = BeautifulSoup(page_source, 'html.parser')

  nome_professor = soup.find('div', class_='ficha-campo caps maior cor-texto')

  first_button_id = 'proj-sup'

  link = []
  nome = []
  inicio_vigencia = []
  fim_vigencia = []
  carga_horaria = []
  professor = []
  resume = []
  enfase = []
  ativo = []
  try:

        first_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, first_button_id))
        )
        driver.execute_script("arguments[0].scrollIntoView();", first_button)
        first_button.click()

        sleep(3)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        div_table = soup.find('div', id='projetos')

        table_soup = div_table.find('table', class_='tabela-dados cor-borda')

        td_info = table_soup.find_all('td')

        counter = 0

        for i in range(len(td_info)):

          if counter == 0:
            link.append(url_base + (td_info[i].find('a').get('href')))
            aux_enfase, aux_resume = get_informacao(url_base + (td_info[i].find('a').get('href')))
            nome.append(td_info[i].find('a').text)
            enfase.append(aux_enfase.upper())
            resume.append(aux_resume)
            professor.append(nome_professor.text)
          elif counter == 1:
              inicio_vigencia.append(td_info[i].text)
          elif counter == 2:
                fim_vigencia.append(td_info[i].text)
                data_comparacao = datetime.strptime(td_info[i].text, '%d/%m/%Y')
                data_atual = datetime.now()
                if data_comparacao >= data_atual:
                  ativo.append('Sim')
                else:
                  ativo.append('Não')
          elif counter == 3:
               carga_horaria.append(td_info[i].text)

          counter = counter + 1

          if counter == 4:
            counter = 0

  except TimeoutException:
        print("Timeout: Um dos botões não ficou disponível para clique.")
  except ElementNotInteractableException:
        print("Erro: Um dos botões não está interagível no momento.")

  dados = {
      'professor': professor,
      'nome': nome,
      'inicio_vigencia': inicio_vigencia,
      'fim_vigencia': fim_vigencia,
      'carga_horaria': carga_horaria,
      'link': link,
      'resume': resume,
      'enfase': enfase,
      'ativo': ativo
    }

  df = pd.DataFrame(dados)
  return df


##Testes

###Teste coleta das URLS de cada curso

In [ ]:
url_acess = get_url(url)

print(url_acess)

['https://institucional.ufpel.edu.br/cursos/cod/5700', 'https://institucional.ufpel.edu.br/cursos/cod/3900', 'https://institucional.ufpel.edu.br/cursos/cod/3910', 'https://institucional.ufpel.edu.br/cursos/cod/6100', 'https://institucional.ufpel.edu.br/cursos/cod/6400']


###Teste coleta das URLS de cada professor dos cursos

In [ ]:
print(url_acess[0])
acesso_professor = get_professor(url_acess[0])

print(acesso_professor)

https://institucional.ufpel.edu.br/cursos/cod/5700
['https://institucional.ufpel.edu.br/servidores/id/33388', 'https://institucional.ufpel.edu.br/servidores/id/26810', 'https://institucional.ufpel.edu.br/servidores/id/163443', 'https://institucional.ufpel.edu.br/servidores/id/28492', 'https://institucional.ufpel.edu.br/servidores/id/32787', 'https://institucional.ufpel.edu.br/servidores/id/22452', 'https://institucional.ufpel.edu.br/servidores/id/34464', 'https://institucional.ufpel.edu.br/servidores/id/35906', 'https://institucional.ufpel.edu.br/servidores/id/16200', 'https://institucional.ufpel.edu.br/servidores/id/28310', 'https://institucional.ufpel.edu.br/servidores/id/28321', 'https://institucional.ufpel.edu.br/servidores/id/27313', 'https://institucional.ufpel.edu.br/servidores/id/32463', 'https://institucional.ufpel.edu.br/servidores/id/17403', 'https://institucional.ufpel.edu.br/servidores/id/26077', 'https://institucional.ufpel.edu.br/servidores/id/66175', 'https://institucio

###Teste coleta das informações dos projetos de cada professor

In [ ]:
verify_info = get_info_professor('https://institucional.ufpel.edu.br/servidores/id/33388')

print(verify_info)

                       professor  \
0   ALAN JOHN ALEXANDER MCBRIDE    
1   ALAN JOHN ALEXANDER MCBRIDE    
2   ALAN JOHN ALEXANDER MCBRIDE    
3   ALAN JOHN ALEXANDER MCBRIDE    
4   ALAN JOHN ALEXANDER MCBRIDE    
5   ALAN JOHN ALEXANDER MCBRIDE    
6   ALAN JOHN ALEXANDER MCBRIDE    
7   ALAN JOHN ALEXANDER MCBRIDE    
8   ALAN JOHN ALEXANDER MCBRIDE    
9   ALAN JOHN ALEXANDER MCBRIDE    
10  ALAN JOHN ALEXANDER MCBRIDE    
11  ALAN JOHN ALEXANDER MCBRIDE    
12  ALAN JOHN ALEXANDER MCBRIDE    

                                                 nome inicio_vigencia  \
0   Utilização de esporos de Bacillus spp. como si...      01/01/2020   
1   Vacinologia Reversa: localização de proteínas ...      01/01/2018   
2   Avaliação de novos alvos vacinais como vacina ...      01/02/2019   
3   Vacina recombinante contra leptospirose: avali...      01/10/2019   
4   Vacina recombinante contra leptospirose: compl...      03/06/2019   
5   Aprimoramento de uma vacina universal contra L...    

##Extração

In [ ]:
df_final = pd.DataFrame()

url_acess = get_url(url)

for url in url_acess:
  professor = get_professor(url)
  for projeto in professor:
    df_temp = get_info_professor(projeto)
    df_final = pd.concat([df_final, df_temp], ignore_index=True)

df_final.to_csv('projetos.csv', index=False)
df_final.to_excel('projetos.xlsx', index=False, engine='openpyxl')

Timeout: Um dos botões não ficou disponível para clique.
Timeout: Um dos botões não ficou disponível para clique.
Timeout: Um dos botões não ficou disponível para clique.
Timeout: Um dos botões não ficou disponível para clique.
Timeout: Um dos botões não ficou disponível para clique.
Timeout: Um dos botões não ficou disponível para clique.
Timeout: Um dos botões não ficou disponível para clique.
Timeout: Um dos botões não ficou disponível para clique.
Timeout: Um dos botões não ficou disponível para clique.
Timeout: Um dos botões não ficou disponível para clique.


In [ ]:
print(df_final[0:4])

                      professor  \
0  ALAN JOHN ALEXANDER MCBRIDE    
1  ALAN JOHN ALEXANDER MCBRIDE    
2  ALAN JOHN ALEXANDER MCBRIDE    
3  ALAN JOHN ALEXANDER MCBRIDE    

                                                nome inicio_vigencia  \
0  Utilização de esporos de Bacillus spp. como si...      01/01/2020   
1  Vacinologia Reversa: localização de proteínas ...      01/01/2018   
2  Avaliação de novos alvos vacinais como vacina ...      01/02/2019   
3  Vacina recombinante contra leptospirose: avali...      01/10/2019   

  fim_vigencia carga_horaria  \
0   01/01/2023             1   
1   01/01/2023             2   
2   31/12/2021             2   
3   30/09/2021             1   

                                                link  \
0  https://institucional.ufpel.edu.br/projetos/id...   
1  https://institucional.ufpel.edu.br/projetos/id...   
2  https://institucional.ufpel.edu.br/projetos/id...   
3  https://institucional.ufpel.edu.br/projetos/id...   

                     